# Modelo de Mercado (Unifactorial)


El modelo de de mercado unifactorial consiste en tratar de explicar la rentabilidad de un activo a través del comportamiento del mercado, medido a través de la evolución de un indice. Es decir, se trata de realizar una regresión lineal unifactorial entre el comportamiento del indice y el comportamiento del mercado.

$$\tilde{R}_{it}=\alpha_i+\beta_i \tilde{R}_{Mt}+\tilde{\varepsilon}_{it}$$

La rentabilidad depende de tres componentes:
- $\alpha_i$: la parte del riesgo explicada por determinadas características del riesgo que no se deben a la evolución de la economía sino a características predecibles del título, se supone constante en el tiempo.
- $\beta_i$:la parte del riesgo que obedece a la evolución general de la economía que vienen recogido por el índice de mercado (RIESGO SISTEMÁTICO)
- $\varepsilon_{it}$:perturbación aleatoria del título, provocado por las decisiones o acontecimientos no predecibles (RIESGO ESPECÍFICO)

## Hipótesis del modelo

- El riesgo específico es una variable aleatoria con $E(\tilde{\varepsilon}_{it})=0$. Esto tiene sentido,cualquir componente predecible debe estar incluido en $\alpha_i$.
- El riesgo específico y el sistemático deben estar incorrelacionados $cov(R_{Mt},\varepsilon_{it}), \forall t$.
- No debe existir correlación entre los riesgos específicos de dos activos distintos $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt}), \forall t$.
- No debe existir correlación entre los errores a lo largo del tiempo  $cov(\tilde{\varepsilon}_{it},\tilde{\varepsilon}_{jt'}),\forall t\neq t'$
- La varianza debe del riesgo específico ser constante a lo largo del tiempo $\sigma^2(\tilde{\varepsilon}_{it})=\sigma^2(\tilde{\varepsilon}_{it'}),\forall t\neq t'$.
- El riesgo sistemático se comporta como $\tilde{\varepsilon}_{it}=N(0,1),\forall t$

In [43]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import numpy as np

In [44]:
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2014-01-01'
END_DATE = '2018-12-31'

In [45]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,)['Adj Close']
df.head()

[*********************100%***********************]  2 of 2 completed


,AMZN,^GSPC
Date,,
2014-01-02,19.898500,1831.979980
2014-01-03,19.822001,1831.369995
2014-01-06,19.681499,1826.770020
2014-01-07,19.901501,1837.880005
2014-01-08,20.096001,1837.489990


Tenemos los datos diarios y queremos trabajar con datos mensuales. Para ello generamos un nuevo dataframe `X` y utilizamos la función `resample('M').last()` tomando como referencia el último día del mes con cotización.

In [46]:
df.index = pd.to_datetime(df.index)
X=df.resample('M').last()
X.head()

,AMZN,^GSPC
Date,,
2014-01-31,17.934500,1782.589966
2014-02-28,18.105000,1859.449951
2014-03-31,16.818501,1872.339966
2014-04-30,15.206500,1883.949951
2014-05-31,15.627500,1923.569946


In [47]:
monthly_ret = np.log(X).diff().dropna()
monthly_ret.head()

,AMZN,^GSPC
Date,,
2014-02-28,0.009462,0.042213
2014-03-31,-0.073709,0.006908
2014-04-30,-0.100757,0.006182
2014-05-31,0.027309,0.020812
2014-06-30,0.038384,0.018879


Calculamos la $\beta$ utilizando las varianza:
$$\beta=\frac{cov(AMZN,GSPC)}{\sigma^2(GSPC)}$$

In [48]:
covariance = monthly_ret.cov().iloc[0,1]
benchmark_variance = monthly_ret.cov().iloc[1,1]
beta = covariance / benchmark_variance
beta

1.6106258640720965

Estimamos la beta utilizando una regresión lineal para ello:
1. `y = monthly_ret.pop('AMZN')`: extraemos los datos de `AMZN`en una serie que denominamos `y`.
2. `monthly_ret=sm.add_constant(monthly_ret)`:añadimos una fila de unos en el dataframe `monthly_ret`.
3. `capm_model = sm.OLS(y, monthly_ret).fit()`: realizamos la regresión por mínimos cuadrados ordinarios.

In [49]:
y = monthly_ret.pop('AMZN')
y.head()

Date
2014-02-28    0.009462
2014-03-31   -0.073709
2014-04-30   -0.100757
2014-05-31    0.027309
2014-06-30    0.038384
Freq: M, Name: AMZN, dtype: float64

In [52]:
type(y)

pandas.core.series.Series

In [50]:
monthly_ret=sm.add_constant(monthly_ret)
monthly_ret.head()

,const,^GSPC
Date,,
2014-02-28,1.0,0.042213
2014-03-31,1.0,0.006908
2014-04-30,1.0,0.006182
2014-05-31,1.0,0.020812
2014-06-30,1.0,0.018879


In [51]:
capm_model = sm.OLS(y, monthly_ret).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                   AMZN   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     34.78
Date:                Sun, 05 Nov 2023   Prob (F-statistic):           2.11e-07
Time:                        19:22:59   Log-Likelihood:                 77.379
No. Observations:                  59   AIC:                            -150.8
Df Residuals:                      57   BIC:                            -146.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.009      1.701      0.0